<h1> What is the Top Complaint Type? </h1>
    
The goal of this exercise is to find the answer to the Question 1 of the problem statement:

<b>Which type of complaint should the Department of Housing Preservation and Development of New York City focus on first?</b>

In this exercise, you need to read back the 311 datasets that you stored in Cloud Object Store and explore the dataset.

By the end of this exercise, you need to figure out the correct Complaint Type that the Department of Housing Preservation and Development of New York City should focus on.

Add the answer to this question along with code and comments in a notebook. Upload the notebook in the subsection called "Question 1 - Which type of complaints The Department of Housing Preservation and Development of New York City should focus first?"  in the section "Submit your work and Review your Peer's work' in module 'Submit Your Work and Grade Your Peers".


<b>Step 1</b>: Download the 311 data from Object Store and install needed libraries

In [1]:
#IMPORT THE LIBRARIES

import pandas as pd
import matplotlib.pylab as plt

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import os

In [2]:
#FUNCTIONS DEFINED TO UPLOAD/DOWNLOAD TO OBJECT STORE

from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded : ' + local_file_name)

from ibm_botocore.client import Config
import ibm_boto3

def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [4]:
#DOWNLOAD THE FILE USING THE DEFINED FUNCTIONS ABOVE
download_file_cos(credentials,'311_ALL.pkl','311_ALL.pkl')

File Downloaded


In [5]:
#DEBUG : Make sure file is there
#!ls

In [6]:
#READ IN THE PICKLE FILE THAT WAS DOWNLOADED
df_311 = pd.read_pickle('311_ALL.pkl')

In [7]:
#DEBUG : Explore the data
#df_311.head()

<b>Step 2</b> : Clense the Data and Remove Duplicate Calls

In [8]:
#CONVERT EVERYTHING TO UPPERCASE AND REMOVE ANY POSSIBLE SPACES IN FRONT OR BACK OF THE DATA
df_311['complaint_type'] = df_311['complaint_type'].str.upper() 
df_311['complaint_type'] = df_311['complaint_type'].str.strip()

df_311['incident_address'] = df_311['incident_address'].str.upper() 
df_311['incident_address'] = df_311['incident_address'].str.strip()

df_311['city'] = df_311['city'].str.upper() 
df_311['city'] = df_311['city'].str.strip()

#MAKE SURE THAT THE COMPLAINT_TYPES ALL MATCH. PAINT/PLASTER AND PAINT - PLASTER ARE THE SAME THING, SO WE WILL MAKE THEM A SINGLE CATAGORICAL VALUE
df_311['complaint_type'] = df_311['complaint_type'].str.replace('PAINT/PLASTER','PAINT - PLASTER')

In [9]:
#DEBUG: VIEW THE DATAFRAME
#df_311.head(4)

In [10]:
#Optional : Set to make sure that the full data in columns can be displayed.
#pd.set_option('display.max_colwidth', -1)

In [11]:
df_311_Duplicate = df_311

In [12]:
# ADD A BOOLEAN TO DETECT DUPLICATES. TODO: MAKE THIS MORE EFFCIENT (WHY DO I HAVE TO CAST???)
df_311_Duplicate['is_duplicate'] = df_311['resolution_description'].str.contains('duplicate').astype(bool)

In [13]:
#DEBUG: VIEW THE DATAFRAME
#df_311_Duplicate.head(4)

In [14]:
#FILTETER OUT ANYTHING THAT ISN'T A DUPLICATE CALL
df_311_Not_Duplicated=df_311_Duplicate[~df_311_Duplicate['is_duplicate']]

In [15]:
#FINAL DATASET THAT DOES NOT CONTAIN DUPLICATES
#df_311_Not_Duplicated.head(4)

<b>Step 3</b> : Let's do some exploration. We have two datasets. df_311_Duplicate contains ALL the calls, including duplicates. df_311_Not_Duplicated does NOT contain the duplicate calls. To answer the question "What is the top complaint" can vary. e.g.,:

1. If I am looking to staff people to address complaints, duplicates wouldn't matter as much. Once the item is fixed, the duplicates go away, and I only need one person to fix it. In this case, I would analyze without duplicates.

2. If I am looking for wich complaints the most people are mad about, and thus making them happy, I would use duplicates. 

The question is, are they the same?

In [16]:
#ADD A COUNT COLUMN, SO THAT WE CAN EASILY DO A COUNT USING GROUP BY LATER

df_311_Duplicate['count'] = 1
df_311_Not_Duplicated['count'] = 1

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
#CREATE A DATASET GROUPED BY COMPLAINT TYPE AND ADD THE COUNTS
df_311_Not_Duplicated_group = df_311_Not_Duplicated.groupby(['complaint_type']).count()['count']

In [18]:
#RESET THE INDEX JUST IN CASE
df_311_Not_Duplicated_group = df_311_Not_Duplicated_group.reset_index()

In [24]:
df_311_Not_Duplicated_group.sort_values(by=['count']).tail(5)

,complaint_type,count
8,GENERAL CONSTRUCTION,500728
15,PLUMBING,695431
14,PAINT - PLASTER,702978
9,HEAT/HOT WATER,738986
10,HEATING,886851


In [20]:
df_311_Duplicate_group = df_311_Duplicate.groupby(['complaint_type']).count()['count']


In [21]:
df_311_Duplicate_group = df_311_Duplicate_group.reset_index()

In [23]:
df_311_Duplicate_group.sort_values(by=['count']).tail(5)

,complaint_type,count
8,GENERAL CONSTRUCTION,500863
15,PAINT - PLASTER,703051
16,PLUMBING,703648
10,HEATING,887869
9,HEAT/HOT WATER,1152592


"Question 1 - Which type of complaints The Department of Housing Preservation and Development of New York City should focus first?" 

<b>ANSWER:</b> While they are in slight different orders, you can see the top 5 in both situations (duplicate vs non-duplicate calls) are:

UNSANITARY CONDITION <p>
GENERAL CONSTRUCTION <p>
PLUMBING <p>
PAINT - PLASTER <p>
HEAT/HOT WATER <p>
HEATING <p>